In [47]:
import numpy as np
from scipy.stats import invwishart

In [11]:
np.random.seed(42)

In [12]:
def generate_samples(n):
    # size = nx2
    mean = np.array([0,0])
    cov = np.array([[1,0],[0,2]])

    samples = np.random.multivariate_normal(mean,cov,size=n)
    return samples

In [13]:
n10 = generate_samples(10)
n100 = generate_samples(100)
n1000 = generate_samples(1000)

In [14]:
def mle_estimate(samples):
    return samples.T@samples/len(samples)

In [ ]:
def bayesian_estimate_invwishart_cp(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[4,0],[0,5]])
    d = 1
    dof = n + v0
    scale = delta0 + samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

In [ ]:
def bayesian_estimate_jeffery(samples):
    # https://tminka.github.io/papers/minka-gaussian.pdf
    n = len(samples)
    d = 1
    dof = n
    scale = samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

def bayesian_estimate_ind_jeffery(samples):
    # gelman page 72
    n = len(samples)
    d = 1
    dof = n - 1
    scale = samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

In [75]:
def mc_bayesian_a(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[4,0],[0,5]])

    for m in [1e3, 1e4, 1e5]:
        numerators = []
        denominators = []
        for _ in range(int(m)):
            sigma_j = invwishart.rvs(v0,delta0)
            det_term = np.linalg.det(sigma_j)**(-n/2)
            exp_term = np.exp(-0.5*np.linalg.trace(np.linalg.inv(sigma_j)@(samples.T@samples)))
            numerators.append(sigma_j*det_term*exp_term)
            denominators.append(det_term*exp_term)

        print(f"For m={m}:\n")
        print(np.array(numerators).mean(axis=0)/np.mean(denominators))

def mc_bayesian_b(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[2,0],[0,4]])

    for m in [1e3, 1e4, 1e5]:
        numerators = []
        denominators = []
        for _ in range(int(m)):
            sigma_j = invwishart.rvs(v0,delta0)
            det_term = np.linalg.det(sigma_j)**(-n/2)
            exp_term = np.exp(-0.5*np.linalg.trace(np.linalg.inv(sigma_j)@(samples.T@samples)))
            numerators.append(sigma_j*det_term*exp_term)
            denominators.append(det_term*exp_term)

        print(f"For m={m}:\n")
        print(np.array(numerators).mean(axis=0)/np.mean(denominators))
